In [2]:
import numpy as np
import random
import sympy as sym
import winsound
from joblib import Parallel, delayed
from sympy import *
from sympy import symbols
from sympy.plotting import plot, plot3d
from scipy.stats import beta
from datetime import datetime
import winsound

In [3]:
duration = 1000  # milliseconds
freq = 440  # Hz

In [36]:
def Pr(xt_,yt,c,param):
    print(yt.shape)
    alpha_=param[0]
    beta_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4] 
    
    pow0=np.count_nonzero(yt == 0)  
    pow1=np.count_nonzero(yt == 1)
    print(pow0,pow1)
    p0=(1-theta_0_)**pow0*(theta_0_)**pow1
    p1=(1-theta_1_)**pow0*theta_1_**pow1
    
    #p0=(theta_0_)**pow1
    #p1=theta_1_**pow1
    
    if xt_==0:
        p0=p0*(1-alpha_-beta_*c)
        p1=p1*(alpha_+beta_*c)
    else:
        p0=p0*gama_
        p1=p1*(1-gama_)

      
    return p0,p1
                

In [5]:
# function to count the number of the infected neighbores of i at t:
def CNbr(G,X,n,T):
    C=np.zeros((T,n))
    for t in range(T):
        C[t]=G[t].dot(X.T[t])
    return C.T

In [6]:
def Forward_Sampling(T,n,G,Y,param):
    
    alpha_=param[0]
    bata_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4] 
    X=np.zeros((n,T))  
    
    for t in range(1,T):
            
        c=G[t-1].dot(X.T[t-1])
        
        for i in range(n):
                    
            pr=Pr(X[i,t-1],Y[:,i,t],c[i],param)
            p0=pr[0]
            p1=pr[1]
                    
            if p0+p1==0:
                l=0.5
            else:
                l=p1/(p0+p1)
                        
            X[i,t]= np.random.binomial( 1, l,size=None)
                    
        
    return X

In [7]:
def Sample_hidden_state(T,n,X,G,Y,param,t):
    
    alpha_=param[0]
    beta_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4]
    
    pow_stay_healthy,pow_become_infected,pow_recovery,pow_stay_infected=0,0,0,0
    
    if (t==0):
        c=G[t].dot(X.T[t])
    else:    
        c=G[t-1].dot(X.T[t-1])
        
    for i in range(n):
        
        alpha_=param[0]
        beta_=param[1]
        gama_=param[2]
        theta_0_=param[3]
        thata_1_=param[4] 
    
        pow0=np.count_nonzero(Y[:,i,t] == 0)  
        pow1=np.count_nonzero(Y[:,i,t] == 1)
    
        p0=(1-theta_0_)**pow0
        p1=(1-theta_1_)**pow0
    
        p0=(theta_0_)**pow1
        p1=theta_1_**pow1
        
        if (X[i,t]==1)&(t==0):
            c0=c[i]-1
            c1=c[i]
        else:
            c0=c[i]
            c1=c[i]+1
        
                
        stay_healthy0= 1-alpha_-beta_*c0
        become_infected0=alpha_+beta_*c0
    
        stay_healthy1=1-alpha_-beta_*c1
        become_infected1=alpha_+beta_*c1
        
        if t==T-1:
            if X[i,t-1]==0:
                p0=p0*stay_healthy0
                p1=p1*become_infected0
            else:
                p0=p0*gama_
                p1=p1*(1-gama_) 
        else:        
            state_transition=X.T[t]+2*X.T[t+1]+1
            key=np.multiply(G[0][i],state_transition)
    
            pow_stay_healthy=pow_stay_healthy+np.count_nonzero(key == 4)
            pow_become_infected=pow_become_infected+np.count_nonzero(key == 3)
            pow_recovery=pow_recovery+np.count_nonzero(key == 2)
            pow_stay_infected=pow_stay_infected+np.count_nonzero(key ==1)
    
            p0=P*p0*(stay_healthy0)**pow_stay_healthy*(become_infected0)**pow_become_infected*(gama_)**pow_recovery*(1-gama_)**pow_stay_infected
            p1=(1-P)*p1*(stay_healthy1)**pow_stay_healthy*(become_infected1)**pow_become_infected*(gama_)**pow_recovery*(1-gama_)**pow_stay_infected
            
        if p0+p1==0:
            l=0.5
        else:
            l=p1/(p0+p1)
        if (l>1 )| (l==0) | (l<0 ):
            print(i,t,p0,p1,stay_healthy0,become_infected0,c0)
        X[i,0]=np.random.binomial( 1,  l,size=None)    
        
    return X    
    

In [8]:
# Gibbs sampling to generate J samples of X:
def Calculate_X(J,T,n,X,G,Y,param):

    alpha_=param[0]
    beta_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4]
    
    for j in range(J):
        for t in range(T):
            X=Sample_hidden_state(T,n,X,G,Y,param,t)
    return X                

In [9]:
# function to sample new parameters and update parameters:
def Params(R,G,X,n,T,Y,param):
    
    alpha_=param[0]
    bata_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4]    
   
    TP=np.sum(np.multiply(X,Y))/6
    FP=n*T-np.count_nonzero(X-Y+1)/6
    
    infR=np.array(CNbr(G,X,n,T))
    
    alpha_=Sample_alpha(a_alpha + n*T- np.count_nonzero(R) , b_alpha - np.count_nonzero(X)+ np.count_nonzero(R))
    beta_=Sample_beta(a_beta + n*T-np.count_nonzero(R-2) , b_beta +np.sum(np.multiply((1-X),infR))-n*T+np.count_nonzero(R-2))
    gama_=Sample_gama(a_gama +(T-1)*n-np.count_nonzero(X[:,:-1]-X[:,1:]-1), b_gama+np.sum(X)-(T-1)*n+np.count_nonzero(X[:,:-1]-X[:,1:]-1))
    theta_0_=Sample_theta0( a_teta0+FP,b_teta0+n*T-np.count_nonzero(X)-FP)
    theta_1_=Sample_theta1( a_teta1+TP,b_teta1+np.count_nonzero(X)-TP)
    
    R=np.zeros((n,T))+1
    for i in range(n):
        for t in range(T-1):
            infr=int(infR[i,t])
            pr_a=alpha_/(alpha_+beta_*infr)
            pr_b=beta_/(alpha_+beta_*infr)
            v=np.random.multinomial(1, [pr_a]+[pr_b]*infr)
            if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2

    return alpha_,beta_,gama_,theta_0_,theta_1_,R

# function to sample from beta distribution

In [10]:
def Sample_alpha(a_alpha, b_alpha):
    for i in beta.rvs(a_alpha, b_alpha, size=1000):
        if (i>0.001)&(i<0.051):
            alpha_=round(i,3)
            break
    return alpha_        

In [11]:
def Sample_beta(a_beta, b_beta):
    for i in beta.rvs(a_beta, b_beta, size=1000):
        if (i>0.0001)&(i<0.051):
            beta_=round(i,4)
            break
    return beta_        

In [12]:
def Sample_gama(a_gama,b_gama):
    for i in beta.rvs(a_gama, b_gama, size=1000):
        if (i>0.1)&(i<0.7):
            gama_=round(i,3)
            break
    return gama_  

In [13]:
def Sample_theta0(a_teta0, b_teta0):
    for i in beta.rvs(a_teta0, b_teta0, size=10000):
        if (i>0.001)&(i<0.3):
            theta_0_=round(i,3)
            break
    return theta_0_  

In [14]:
def Sample_theta1(a_teta1, b_teta1):
    for i in beta.rvs(a_teta1, b_teta1, size=10000):
        if i>0.8:
            theta_1_=round(i,3)
            break
    return theta_1_  

In [15]:
# initialize parameters for beta distributions:
a_alpha=0.1
b_alpha=3000
a_beta=0.1
b_beta=1
a_gama=7
b_gama=7
a_teta0=1
b_teta0=5
a_teta1=5000
b_teta1=.1

In [16]:
#Sample infection and emision parameters(alpha,beta,gama,teta0,teta1)
alpha_=Sample_alpha(a_alpha, b_alpha)
beta_=Sample_beta(a_beta, b_beta)
gama_=Sample_gama(a_gama,b_gama)
theta_0_=Sample_theta0(a_teta0, b_teta0)
theta_1_=Sample_theta1(a_teta1, b_teta1)

In [17]:
param=[alpha_,beta_,gama_,theta_0_,theta_1_]
param

[0.001, 0.0132, 0.479, 0.088, 1.0]

In [37]:
# load data sets for Y,G and generate first J samples of X:
%store -r Y
%store -r G

U=1
T=G.shape[0]
n=G.shape[1]
J=1
P=0.9
#Y=np.logical_or(np.logical_or(np.logical_or(Y[0],Y[1]),np.logical_or(Y[2],Y[3])),np.logical_or(Y[4],Y[5]))+0
param=[alpha_,beta_,gama_,theta_0_,theta_1_]
X=Forward_Sampling(T,n,G,Y,param)    
X=Calculate_X(J,T,n,X,G,Y,param)
winsound.Beep(freq, duration)   

(6,)
4 2
(6,)
5 1
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
4 2
(6,)
5 1
(6,)
6 0
(6,)
4 2
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(

5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
2 4
(6,)
6 0
(6,)
2 4
(6,)
5 1
(6,)
5 1
(6,)
5 1
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
5 1
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
2 4
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
2 4
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6

(6,)
6 0
(6,)
3 3
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
4 2
(6,)
4 2
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
4 2
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
3 3
(6,)
6 0
(6,)
2 4
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
3 3
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
3 3
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
3 3
(6,)
2 4
(6,)
4 2
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
5 1
(6,)
5 1
(

6 0
(6,)
5 1
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
2 4
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
4 2
(6,)
4 2
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
6 0
(6,)
5 1
(6,)
5

In [31]:
Y.shape

(6, 63, 66)

In [32]:
# define auxiliary variable R(n,t):
R=np.zeros((n,T))+1
infR=np.array(CNbr(G,X,n,T))
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        pr_a=alpha_/(alpha_+beta_*infr)
        pr_b=beta_/(alpha_+beta_*infr)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr)
        if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2

In [33]:
# Main code to run entire Gibbs algorithm U times:
for i in range(U):
    print("******************* Iteration:",i,"*****************************************************************************")
    prm=Params(R,G,X,n,T,Y,param)
    param=[prm[0],prm[1],prm[2],prm[3],prm[4]]
    X=Calculate_X(J,T,n,X,G,Y,param)
    print("Verctor of Health States:","\n",X)
    R=prm[5]
    print("params:",param)
winsound.Beep(freq, duration)            

******************* Iteration: 0 *****************************************************************************
Verctor of Health States: 
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
params: [0.001, 0.0173, 0.629, 0.065, 1.0]
******************* Iteration: 1 *****************************************************************************
Verctor of Health States: 
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
params: [0.001, 0.0022, 0.544, 0.054, 1.0]
******************* Iteration: 2 *****************************************************************************
Verctor of Health States: 
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
params: [0.001, 0.0084, 0.611

In [34]:
np.sum(Y)

1525.0

In [35]:
np.sum(X)

5.0

In [ ]:
np.sum(Y)

In [ ]:
alpha_

In [ ]:
Y_

In [ ]:
param=[0.007, 0.0003, 0.368, 0.043, 0.976]

In [ ]:
###############################################################################################################################

In [ ]:
#data simulation

In [ ]:
prm=param
alpha_,beta_,gama_,theta_0_,theta_1_=prm[0]+0.003,prm[1]+0.0007,prm[2]-.068,np.round(prm[3]-.003,2),prm[4]-.076
print("alpha:",alpha_)
print("beta:",beta_)
print("gama:",gama_)
print("theta0:",theta_0_)
print("theta1:",theta_1_)

In [ ]:
# generate synthetized G
n,T=100,100
x=int((1-P)*n)
y=10
X=np.zeros((n,T))
idx=np.random.choice(range(n), x)
X[idx,0]=1
G=[]
for j in range(T):
    g=np.identity(n)
    for i in range(n):
        inx=np.random.choice(range(i,n), y)
        g[i,inx]=1  
        g[inx,i]=1
    G.append(g)

In [ ]:
np.sum(G[0])

In [ ]:
np.sum(X.T[0])

In [ ]:
infR=CNbr(G,X,n,T)

In [ ]:
infR

In [ ]:
infR[i,0]

In [ ]:
# Calculate X from G and params
for t in range(T-1):
    for i in range(n):
        c=infR[i,t]
        if X[i,t]==0:
            p0=(1-alpha_-beta_*c)
            p1=alpha_+beta_*c
        else:
            p0=gama_
            p1=1-gama_
        if p0+p1==0:
            l=0.5
        else:
            l=p1/(p0+p1)
        X[i,t+1]=np.random.binomial( 1, l,size=None) 


In [ ]:
X

In [ ]:
np.sum(X)

In [ ]:
x

In [ ]:
# Generate synthetized Y
Y1=np.zeros((n,T))
for t in range(T):
    for i in range(n):
        if X[i,t]==0:
            Y1[i,t]=np.random.binomial( 1, theta_0_,size=None) 
        else:
            Y1[i,t]=np.random.binomial( 1, theta_1_,size=None) 

In [ ]:
np.sum(Y1)

In [ ]:
np.sum(Y1-Y)

In [ ]:
Y.shape

In [ ]:
################################################################################################################################

In [ ]:
# test the code with simulation data

In [ ]:
from datetime import datetime

In [ ]:
# initialize parameters for beta distributions:
a_alpha=0.1
b_alpha=3000
a_beta=0.1
b_beta=1
a_gama=7
b_gama=7
a_teta0=1
b_teta0=5
a_teta1=5000
b_teta1=.1

In [ ]:
#Sample infection and emision parameters(alpha,beta,gama,teta0,teta1)
alpha_=0
alpha_=Sample_alpha(a_alpha, b_alpha)
beta_=Sample_beta(a_beta, b_beta)
gama_=Sample_gama(a_gama,b_gama)
theta_0_=Sample_theta0(a_teta0, b_teta0)
theta_1_=Sample_theta1(a_teta1, b_teta1)

In [ ]:
param=[alpha_,beta_,gama_,theta_0_,theta_1_]
param

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time is :", current_time)

In [ ]:
# define auxiliary variable R(n,t):
R=np.zeros((n,T))+1
infR=np.array(CNbr(G,X,n,T))
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        pr_a=alpha_/(alpha_+beta_*infr)
        pr_b=beta_/(alpha_+beta_*infr)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr)
        if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2

In [ ]:
# Main code to run entire Gibbs algorithm U times:
for i in range(U):
    print("******************* Iteration:",i,"*****************************************************************************")
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time is :", current_time)    
    prm=Params(R,G,X,n,T,Y,param)
    param=[prm[0],prm[1],prm[2],prm[3],prm[4]]
    X=Calculate_X(J,T,n,X,G,Y,param)
    print("Verctor of Health States:","\n",X)
    R=prm[5]
    print("params:",param)
winsound.Beep(freq, duration)            

In [ ]:
print("hidden states:",X)

In [ ]:
prm=param
alpha_,beta_,gama_,theta_0_,theta_1_=prm[0]+0.003,prm[1]+0.0007,prm[2]-.068,np.round(prm[3]-.003,2),prm[4]-.076
print("alpha:",alpha_)
print("beta:",beta_)
print("gama:",gama_)
print("theta0:",theta_0_)
print("theta1:",theta_1_)

In [ ]:
#add noise to data

In [ ]:
mask = np.random.binomial(1, 0.05,X.shape[1]).astype(bool)
indx=np.random.choice(range(n), 30)
for i in indx:
    x=X[i]
    x[mask] = 1 - x[mask]
    X[i]=x